In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [1]:
def process_cpi_chained(input_path, output_csv):
    """
    Clean and transform raw CPI Excel data into a usable monthly inflation dataset.

    This function reads a CPI Excel file (for eggs, coffee, bread, etc.), removes messy header rows, 
    converts each month's values to numbers, reshapes the data from wide format (each month in its own column)
    into long format (one row per month), creates a proper date column, calculates month-to-month inflation, and saves the cleaned result as a CSV file.

    Parameters:
    input_path : str
        File path to the raw CPI Excel file.
    output_csv : str
        File path to save the cleaned CSV output.

    Returns:
    pandas.DataFrame
        A cleaned DataFrame containing Year, Month, CPI value, Date, and calculated Inflation.
    """

    # Load Excel file and skip text rows
    raw_data = pd.read_excel(input_path, skiprows=7, header=None)

    # Extract the real header row, clean whitespace
    header_row = raw_data.iloc[2].astype(str).str.strip()
    raw_data.columns = header_row

    # Drop extra top rows and reset indexing
    raw_data = raw_data.drop(index=[0, 1, 2]).reset_index(drop=True)

    # Keep only rows where the year looks like a number
    raw_data = raw_data[raw_data["Year"].apply(lambda x: str(x).isdigit())]
    raw_data["Year"] = raw_data["Year"].astype(int)

    # The 12 month columns in the CPI sheet
    month_names = ["Jan","Feb","Mar","Apr","May","Jun", "Jul","Aug","Sep","Oct","Nov","Dec"]

    # Convert monthly values to numeric
    raw_data[month_names] = raw_data[month_names].apply(pd.to_numeric, errors="coerce")

    # Convert wide format → long format (one row per month)
    long_format = raw_data.melt(
        id_vars="Year",
        value_vars=month_names,
        var_name="Month",
        value_name="CPI"
    )

    # Convert month names to numbers
    month_to_number = {
        "Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,
        "Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12
    }
    long_format["MonthNum"] = long_format["Month"].map(month_to_number)

    # Create a proper Date column for time-series analysis
    long_format["Date"] = pd.to_datetime(
        dict(year=long_format["Year"], month=long_format["MonthNum"], day=1)
    )

    # Sort data by time
    long_format = long_format.sort_values("Date").reset_index(drop=True)

    # Compute month-to-month inflation %
    long_format["Inflation"] = (
        (long_format["CPI"] - long_format["CPI"].shift(1)) /
         long_format["CPI"].shift(1)
    ) * 100

    # First month has no previous value → set inflation to NaN
    long_format.loc[0, "Inflation"] = np.nan

    # Save cleaned dataset to CSV
    long_format.to_csv(output_csv, index=False)

    return long_format


C:\Users\nikja\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\nikja\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\nikja\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Eggs:
    Year Month    CPI  MonthNum       Date  Inflation
0  2010   Jan  1.789         1 2010-01-01        NaN
1  2010   Feb  1.872         2 2010-02-01   4.639463
2  2010   Mar  1.822         3 2010-03-01  -2.670940
3  2010   Apr  1.779         4 2010-04-01  -2.360044
4  2010   May  1.523         5 2010-05-01 -14.390107 

Coffee:
    Year Month    CPI  MonthNum       Date  Inflation
0  2010   Jan  3.811         1 2010-01-01        NaN
1  2010   Feb  3.736         2 2010-02-01  -1.967987
2  2010   Mar  3.565         3 2010-03-01  -4.577088
3  2010   Apr  3.641         4 2010-04-01   2.131837
4  2010   May  3.664         5 2010-05-01   0.631695 

Bread:
    Year Month    CPI  MonthNum       Date  Inflation
0  2010   Jan  1.360         1 2010-01-01        NaN
1  2010   Feb  1.361         2 2010-02-01   0.073529
2  2010   Mar  1.368         3 2010-03-01   0.514328
3  2010   Apr  1.363         4 2010-04-01  -0.365497
4  2010   May  1.359         5 2010-05-01  -0.293470 

